In [1]:
import pandas as pd
import folium                              # 지도 기반 데이터 시각화
from geopy.distance import great_circle    # 좌표 간 거리를 km 단위로 변환

In [2]:
apt = pd.read_csv('data/seoul_apt.csv', sep=',', encoding='euc-kr')
apt.head(1)

,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,1,A15679103,우리유앤미,아파트,서울특별시 동작구 서달로 83,서울,동작구,흑석동,우리유앤미아파트,서달로,...,위탁,1773.56,223.0,의무,2018-04-10 15:59:42.0,Y,N,126.959639,37.500668,2013-03-07 09:46:59.0


In [3]:
find_apt = "구로한신1차"   # 검색할 아파트

In [4]:
target = apt[apt['k-아파트명'].str.contains(find_apt)]
lat = float(target['좌표Y'])
lon = float(target['좌표X'])
loc = [lat, lon]
loc

[37.4963839, 126.8737607]

In [5]:
fmap = folium.Map(location=loc, zoom_start=16)  # zoom_start : ~18

# 마커 표시
popup = '<pre>' + find_apt + '</pre>'
folium.Marker(loc, tooltip=find_apt, popup=popup).add_to(fmap)

fmap

In [6]:
class CountByWGS84:    # WGS84 위경도 좌표 기반 데이터 집계 및 시각화

    def __init__(self, df, lat, lon, dist=1):
        """
        df: 데이터 프레임
        lat: 중심 위도
        lon: 중심 경도
        dist: 기준 거리(km)
        """
        self.df = df
        self.lat = lat
        self.lon = lon
        self.dist = dist

    def filter_by_rectangle(self):
        """
        사각 범위 내 데이터 필터링
        """
        
        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lon_min = self.lon - 0.015 * self.dist
        lon_max = self.lon + 0.015 * self.dist
        
        self.points = [[lat_min, lon_min], [lat_max, lon_max]]
        
        result = self.df.loc[
            (self.df['lat'] > lat_min) &
            (self.df['lat'] < lat_max) &
            (self.df['lon'] > lon_min) &
            (self.df['lon'] < lon_max)
        ]
        result.index = range(len(result))

        return result

        
    def filter_by_radius(self):
        """
        반경 범위 내 데이터 필터링
        """
        # 사각 범위 내 데이터 필터링
        tmp = self.filter_by_rectangle()

        # 기준 좌표 포인트
        center = (self.lat, self.lon)

        result = pd.DataFrame()
        for index, row in tmp.iterrows():
            # 개별 좌표 포인트
            point = (row['lat'], row['lon'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])

        result.index = range(len(result))

        return result

    def plot_by_radius(self, df):
        """
        반경 범위 내 데이터 플로팅
        """
        for idx, row in df.iterrows():

            lat_ = row['lat']
            lon_ = row['lon']

            folium.Marker(location=[lat_, lon_],
                          radius=15,
                          icon=folium.Icon(color='red', icon='train', icon_color='white', prefix='fa'), 
                          tooltip=row['역사명']).add_to(fmap)

        folium.Circle(radius=dist * 1000,
                      location=[lat, lon],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(fmap)

        return fmap

In [7]:
subway = pd.read_csv('data/subway.csv', sep=',', encoding='euc-kr')
subway = subway.rename(columns={'역위도': 'lat', '역경도': 'lon'})
subway.index = range(len(subway))
subway.head(1)

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,lat,lon,운영기관명,역사도로명주소,역사전화번호,데이터기준일자,제공기관코드,제공기관명
0,3122,부평삼거리,S2801,인천지하철 1호선,Bupyeongsamgeori,富平三거리,일반역,-,-,37.478267,126.710494,인천교통공사,인천광역시 부평구 경인로 765,032-516-3122,2020-11-30,B551894,인천교통공사


In [8]:
dist = 0.5           # 반경 설정 (단위 : km )
df = subway

# 반경 집계 인스턴스 생성
cbw = CountByWGS84(df, lat, lon, dist)

# 사각 범위 내 데이터 필터링
result_rectangle = cbw.filter_by_rectangle()

# 반경 범위 내 데이터 필터링
result_radius = cbw.filter_by_radius()

In [9]:
map = cbw.plot_by_radius(result_radius)
map

In [10]:
map.save('maps/map.html')